# Alessandro's scratch notebook

ICESat-2 hackweek final project 
June 16, 2020  
Alessandro Di Bella

Install extra python modules to
* read polygons from WKT strings
* access Sentinel API (we might want to create a virtual environment to avoid doing this every time)

In [98]:
#!pip install wkt
!pip install sentinelsat

ERROR: Could not find a version that satisfies the requirement wkt (from versions: none)
ERROR: No matching distribution found for wkt


In [100]:
### Clear workspace
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [101]:
### Import modules
import numpy as np
from sentinelsat import SentinelAPI
import shapely.geometry as sg
from astropy.time import Time, TimeDelta
from tqdm import tqdm
import matplotlib.pyplot as plt
import h5py, sys, pyproj
import readers as rd
from icepyx import icesat2data as ipd
import cartopy.crs as ccrs
import importlib
import search_sentinels as ss
import warnings
warnings.filterwarnings('ignore')

### Add path to scripts
if '/home/jovyan/leading_to_phytoplankton/scripts' not in sys.path:
    sys.path.insert(0, '/home/jovyan/leading_to_phytoplankton/scripts')
    
### Magic function to enable interactive plotting
%matplotlib widget 

In [102]:
### Define path to sample data
data_loc = "/home/jovyan/shared/leading_to_phytoplankton/"

In [147]:
# ### Download IS2 sample data
# ## Define search parameters
# short_name = 'ATL03'
spatial_extent = [-180, 50, 180, 90]
# date_range = ['2019-07-26','2019-07-26']
# 
# region_a = ipd.Icesat2Data(short_name, spatial_extent, date_range, \
#                            start_time='21:35:00', end_time='21:36:00', version='002')
# 
# print(region_a.dataset)
# print(region_a.dates)
# print(region_a.start_time)
# print(region_a.end_time)
# print(region_a.dataset_version)
# print(region_a.spatial_extent)
# region_a.visualize_spatial_extent()
# region_a.avail_granules(ids=1)
# 
# ## Log in to EarthData
# earthdata_uid = 'ing.a.dibella'
# email = 'ing.a.dibella@gmail.com'
# region_a.earthdata_login(earthdata_uid, email)
# 
# ## Order granule
# region_a.download_granules(data_loc)

In [104]:
## Load IS2 sample data
f_atl03 = data_loc + "processed_ATL03_20190726213326_04400404_002_01.h5"
f = h5py.File(f_atl03, 'r')
# check to see if it is forward (1)  or backward (0) orientation to know which beam is strong/weak. 
# (2 is transition phase- don't use these data)
print(f['orbit_info/sc_orient'][0])

beam = 'gt2l'
df03 = rd.getATL03(f,beam)
df03.head()
print("Number of point measurements:", len(df03))

0
Number of point measurements: 1068028


In [105]:
### Downsample ATL03 to speed up test
df03 = df03[::100]
print("Number of point measurements:", len(df03))

Number of point measurements: 10681


Pull in some extra ATL03 params and convert GPS to UTC time

In [106]:
### Convert GPS time to UTC time
epoch = f['/ancillary_data/atlas_sdp_gps_epoch'][0]
df03['time'] = Time(epoch + df03['dt'],format='gps').utc.datetime

### Calculate along track distance relative to the beginning of the cut segment
df03['AT_dist'] = df03.x - df03.x.values[0]

### Read orbit number
df03['orbit_number'] = [f['/orbit_info/orbit_number'][0]] * len(df03)
df03.head()

,lats,lons,x,y,heights,dt,conf,time,AT_dist,orbit_number
0,85.679424,37.960135,9606197.0,46.319935,20.501621,4.941210e+07,2,2019-07-26 21:35:00.001129,0.0,4802
100,85.680423,37.953167,9606323.0,46.358212,-2.857844,4.941210e+07,0,2019-07-26 21:35:00.018729,126.0,4802
200,85.681324,37.946888,9606437.0,46.305145,-2.405804,4.941210e+07,0,2019-07-26 21:35:00.034829,240.0,4802
300,85.682385,37.939535,9606570.0,45.949356,6.780809,4.941210e+07,0,2019-07-26 21:35:00.053729,373.0,4802
400,85.683360,37.932681,9606694.0,46.271637,16.332500,4.941210e+07,0,2019-07-26 21:35:00.071029,497.0,4802


In [107]:
### Plot data
var = 'heights' #choose which variable we want to plot
vmin = -10
vmax = 30
ticks = np.arange(-20,100,5)

plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=0)) # choose polar sterographic for projection
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.set_extent([-180, 180, 60, 90], ccrs.PlateCarree())
# plt.scatter(df03['lons'][::1000], df03['lats'][::1000],c=df03[var][::1000], cmap='viridis', vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
plt.scatter(df03['lons'], df03['lats'], c=df03[var], s=1, cmap='viridis', vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
plt.colorbar(label=var, shrink=0.5, ticks=ticks,extend='both')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [108]:
### Random tests to create fix-length strings
cycle = "{:0>2}".format(f['orbit_info/cycle_number'][0])
rgt = "{:0>4}".format(f['orbit_info/rgt'][0])
print("Cycle_number:", cycle)
print("RGT number:", rgt)

Cycle_number: 04
RGT number: 0440


In [152]:
importlib.reload(ss)

<module 'search_sentinels' from '/home/jovyan/leading_to_phytoplankton/scripts/search_sentinels.py'>

In [138]:
type(df03['time'][0])
t_is2       = Time(df03['time'], scale='utc')
dtt         = TimeDelta(3600 * 1, format='sec')
t_is2_start = min(t_is2) - dtt
t_is2_stop  = max(t_is2) + dtt

In [157]:
out = ss.search_sentinels(df03, [-100, 70, 10, 90], dt=3)

Creating AOI polygon...
Selecting orbit data inside AOI...
N. of orbits/points inside AOI: 1/10681
Query for metadata...


Querying products: 100%|██████████| 622/622 [00:09<00:00, 67.86 products/s]
0it [00:00, ?it/s]

N. of total images: 622
Converting time to astropy objects...
Looping over orbits to find intersections within 3h...


1it [00:02,  2.66s/it]
1it [00:00, 75.44it/s]

N. of images within 3h: 622
Creating images footprint polygons...
Looping over orbits to find intersections...
N. of total intersections: 0


In [154]:
out

([], [], [], [], [], {})